In [1]:
from database import get_data

In [2]:
data = get_data()

SELECT id, identificacion, nombre FROM Personas;


In [5]:
data

,id,identificacion,nombre
0,1,12345678,Juan Pérez
1,2,87654321,Maria Gómez
2,3,23456789,J. Pérez
3,4,34567890,José Gómez
4,5,231047465,Ana
...,...,...,...
864,1000,948472764,José
865,1001,135555999,S.
866,1002,703524569,Torres
867,1003,895510194,D. Rossi


In [218]:
# texts = [f"{nombre}{identificacion}" for nombre,  identificacion in zip(data['nombre'], data['identificacion'])]
# texts = data.apply(lambda row: f"{row['nombre']}{row['identificacion']}",axis=1).tolist()


# texts = data['nombre'].tolist()

import re

import unicodedata

# Regex para mantener solo letras y números, excluyendo apóstrofes
_alnum_regex = re.compile(r"(?ui)\W")

def preprocess(s):
    # 1. Normalizar caracteres acentuados a su versión base (ej. á → a, ñ → n)
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("utf-8")
    
    # 2. Eliminar caracteres no alfanuméricos (incluye los apóstrofes)
    string_out = _alnum_regex.sub(" ", s)
    
    # 3. Retornar en minúsculas y sin espacios extras
    return string_out.strip().lower()

# Pruebas

texts =  data['nombre'].apply(lambda x: preprocess(x)).tolist()
ids = data['id'].tolist()



In [219]:
len(texts)

869

In [96]:
# generate embeddings

from sentence_transformers import SentenceTransformer

# Modelo de embeddings local
model = SentenceTransformer('dccuchile/bert-base-spanish-wwm-cased')


No sentence-transformers model found with name dccuchile/bert-base-spanish-wwm-cased. Creating a new one with mean pooling.
Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [200]:
embeddings = model.encode(texts)

In [201]:
len(embeddings)

869

In [202]:
# Index FAISS

import faiss
import numpy as np

# Crear índice y añadir datos
dimension = 768
index = faiss.IndexFlatL2(dimension)

index.add(np.array(embeddings, dtype='float32'))

In [211]:
import pandas as pd
def search(query, threshold=0.1):
    # Generar embedding para la consulta del usuario
    query_embedding = model.encode([preprocess(query)])

    # Buscar en FAISS
    distances, indices = index.search(np.array(query_embedding, dtype='float32'), k=10)
    # Mostrar resultados
    results = []  # Lista para almacenar los resultados
    for i, idx in enumerate(indices[0]):
        # Obtener ID único
        result_id = ids[idx]  
        # Texto original
        result_text = data.iloc[idx]['nombre']
        similarity = 1 - distances[0][i] / max(distances[0])  
        
        # Filtrar resultados por umbral de similitud
        if similarity >= threshold:
            results.append({"ID": result_id, "Registro": result_text, "Similaridad": f"{similarity:.2%}"})

    results_df = pd.DataFrame(results)
    print(results_df)

search("Juan Perez")

    ID      Registro Similaridad
0    1    Juan Pérez     100.00%
1  557   María Pérez      21.46%
2  652   María Pérez      21.46%
3  660    Juan Rossi      18.03%
4  916   Pedro Pérez      16.11%
5   74  Olivia Pérez      13.75%


# Manejar Nuevos Registros / Eliminaciones (mutaciones) sin indexar todo


## Crear nuevo Registro

In [103]:


nuevo_registro = "Guillermo Mendoza"
# ID del nuevo registro
nuevo_id = 1005
# Generar embedding para el nuevo registro
nuevo_embedding = model.encode([nuevo_registro])

# Añadir al índice FAISS
index.add(np.array(nuevo_embedding, dtype='float32'))

# Actualizar listas de IDs y textos
ids.append(nuevo_id)
texts.append(nuevo_registro)

In [134]:
texts[len(texts)-1]

'Guillermo Mendoza'

## Eliminar Registros Existentes

In [170]:

# Supongamos que quieres eliminar el registro con ID 3
id_to_remove = 4

index_to_remove = ids.index(id_to_remove) 

texts[index_to_remove]

'jose gomez'

In [175]:


# Filtrar los registros que quieres mantener
remaining_indices = [i for i, record_id in enumerate(ids) if record_id != id_to_remove]

# Reconstruir el índice FAISS con los registros restantes
new_embeddings = [embeddings[i] for i in remaining_indices]
filtered_embeddings = np.array(new_embeddings, dtype='float32')


index = faiss.IndexFlatL2(dimension)
index.add(filtered_embeddings)

# Actualizar listas de IDs y textos
ids = [ids[i] for i in remaining_indices]
texts = [texts[i] for i in remaining_indices]


In [183]:
search("Jose Gomez")

    ID         Registro Similaridad
0   58    Laura Sánchez     100.00%
1    5         J. Pérez     100.00%
2    2      Maria Gómez      24.79%
3  603          Ramírez      24.79%
4  245          Bianchi      17.63%
5  530         P. Lopez      17.63%
6  846       Juan Weber      17.63%
7  883              S.       13.60%
8  684  Lucas Fernández       7.47%
9  317              D.        0.00%


## Actualizar Registro Existente

por ejemplo corregir un error de nombre o identificacion.
Pasos: 
1. Eliminar el Vector Correspondiente del indice
2. Generar un nuevo embeding y agregarlo en el indice

In [184]:
target_id = 1

target_index = ids.index(target_id)

texts[target_index]

'juan perez'

In [185]:
# Registro actualizado
registro_actualizado = "Juan R. Perez"
# Cambiamos el formato de identificación
id_actualizado = target_id
# Eliminar el registro antiguo
remaining_indices = [i for i, record_id in enumerate(ids) if record_id != id_actualizado]
filtered_embeddings = np.array([embeddings[i] for i in remaining_indices], dtype='float32')

# Reconstruir el índice
index = faiss.IndexFlatL2(dimension)
index.add(filtered_embeddings)

# Añadir el registro actualizado
nuevo_embedding = model.encode([registro_actualizado])
index.add(np.array(nuevo_embedding, dtype='float32'))

# Actualizar listas de IDs y textos
ids = [ids[i] for i in remaining_indices] + [id_actualizado]
texts = [texts[i] for i in remaining_indices] + [registro_actualizado]

### Similitud

initial value 70% (0.70)

In [217]:
search("Maria Gomez",threshold=0.7)

    ID     Registro Similaridad
0    2  Maria Gómez     100.00%
1  602  María Gómez     100.00%
